In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import classification_report
import os

In [24]:
img_width, img_height = 224, 224
batch_size = 16  # Reduced batch size
epochs = 20  # Increased epochs
num_classes = 4

In [3]:
class_names = ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'no_tumor']

In [25]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,  # Added vertical flip
    fill_mode='nearest',
    validation_split=0.2
)

In [26]:
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [27]:
def create_data_generator(data_dir, generator, subset):
    return generator.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        classes=class_names,
        shuffle=True if subset == 'training' else False,
        subset=subset
    )

In [28]:
data_dir = 'Training'

In [29]:
train_generator = create_data_generator(data_dir, train_datagen, 'training')
validation_generator = create_data_generator(data_dir, train_datagen, 'validation')

Found 2297 images belonging to 4 classes.
Found 573 images belonging to 4 classes.


In [35]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [36]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

In [37]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [38]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5)


In [39]:
steps_per_epoch = math.ceil(train_generator.samples / batch_size)
validation_steps = math.ceil(validation_generator.samples / batch_size)

In [40]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping, reduce_lr],
    
)


Epoch 1/20


C:\Users\kusha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 49/144 ━━━━━━━━━━━━━━━━━━━━ 10:12 6s/step - accuracy: 0.3017 - loss: 1.4023

KeyboardInterrupt: 

In [22]:
score = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

17/17 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.5738 - loss: 1.0934
Test loss: 1.1689339876174927
Test accuracy: 0.48713234066963196


In [23]:
validation_generator.reset()
y_pred = model.predict(validation_generator, steps=validation_generator.samples // batch_size + 1)
y_pred_classes = np.argmax(y_pred, axis=1)
true_classes = validation_generator.classes
print(classification_report(true_classes, y_pred_classes, target_names=class_names))

18/18 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step
                  precision    recall  f1-score   support

    glioma_tumor       0.41      0.67      0.51       165
meningioma_tumor       0.31      0.23      0.27       164
 pituitary_tumor       0.65      0.73      0.69       165
        no_tumor       0.00      0.00      0.00        79

        accuracy                           0.47       573
       macro avg       0.34      0.41      0.37       573
    weighted avg       0.40      0.47      0.42       573



C:\Users\kusha\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kusha\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kusha\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
